# AutoEncoder



## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.backend import clear_session
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam

### (2) 필요 함수들 생성

#### 1) reconstruction error plot

> * input과 output의 차이(mse)를 계산하고
* 시각화 합니다.



In [ ]:
def recon_err_plot(x, x_pred, y, threshold=0):
    mse = np.mean(np.power(x - x_pred, 2), axis=1)
    error_df = pd.DataFrame({'Reconstruction_error': mse, 'True_class': y})
    error_df = error_df.reset_index()

    groups = error_df.groupby('True_class')
    fig, ax = plt.subplots(figsize=(15, 8))
    for name, group in groups:
        ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
                label= "Abnormal" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error Plot")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    plt.grid()
    plt.show()

    return error_df

#### 2) precision, recall, f1 curve

> * sklearn에서는 precision, recall curve만 제공됩니다. 
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    plt.figure(figsize = (15,8))
    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')
    plt.xlabel('Anomaly Score')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### 3) threshold로 자르면서 분류 모델 평가


In [ ]:
def classification_report2(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    print('< confusion matrix >\n')
    print(confusion_matrix(y, pred_temp))
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

    return confusion_matrix(y, pred_temp)

#### 4) DL 학습곡선 그리기


In [ ]:
def plot_learning_curve(history) :

    plt.figure(figsize = (12,6))
    plt.plot(history['loss'], label='Train', marker = '.')
    plt.plot(history['val_loss'], label='Validation', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.show()

### (3) 데이터셋 불러오기

![](https://keralakaumudi.com/web-news/en/2020/04/NMAN0141956/image/paper-mill.1.582102.jpg)

In [ ]:
# 공정 데이터 불러오기
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/processminer2.csv"
data = pd.read_csv(path)
data.head()

In [ ]:
date2 = data.copy()
date2['time'] = pd.to_datetime(data.time, format = '%m/%d/%y %H:%M')
date2['day'] = date2['time'].dt.day
date2.loc[date2['y']==1].groupby('day', as_index=False)['y'].count()

## 2.데이터 준비

### (1) 불필요한 변수 제거

불필요한 변수 제거 : time

In [ ]:
data.drop('time', axis=1, inplace=True)

In [ ]:
data.shape

### (2) 데이터 분할

* x, y 분할

In [ ]:
target = 'y'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 6000, random_state = 20)

### (3) 스케일링


In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (4) 학습용 Normal Data

In [ ]:
x_train0 = x_train[y_train == 0]

## 3.모델링 실습
* 원하는 구조로 모델을 설계하고 학습해 봅시다.
* 참조 구성 : 아래와 같이 구성해도 좋습니다.
    * Input(59)
    * Dense(16)
    * Dense(8)
    * Dense(16)
    * Dense(59)


### (1) Auto Encoder
* 모델 설계

* 학습

### (2) 분류에 대한 평가

* 검증 셋으로 예측하고 reconstruction error로 평가해 봅시다.

* thresholds 값을 조절하면서, precision, recall, f1 score 그래프를 그려 봅시다.

* f1 score를 가장 높이는 thresholds 값을 찾고 평가해 봅시다.